In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('HPCG.db')

In [2]:
def extract(filename, data):
    with open(filename, 'r') as file:
        lines = file.readlines()

    NX = ""
    NY = ""
    NZ = ""
    Time = ""
    # Although the time has been set to 1860, it varies
    Gflops = ""

    idd_index = 14
    # the line index of "Local Domain Dimensions="
    T_index = 89
    # the line index of "Benchmark Time Summary::Total=" 
    Gflops_index = 117
    # the line index of "Final Summary="

    NX = lines[idd_index + 1].split("=")[1]
    NY = lines[idd_index + 2].split("=")[1]
    NZ = lines[idd_index + 5].split("=")[1]
    Time = lines[T_index].split("=")[1]
    Gflops = lines[Gflops_index + 1].split("=")[1]

    data.append([NX, NY, NZ, Time, Gflops])
    return data        

In [3]:
import os

# check below before running this script
directory_path = "/home/lizz_lab/cse12111448/HPCG/hpcg/build/bin/"  
core_num = 24

# /home/lizz_lab/cse12111448/HPCG/hpcg/build/bin/HPCG-Benchmark_3.1_2023-11-13_15-36-13.txt
file_names = [file for file in os.listdir(directory_path) if file.startswith("HPCG-Benchmark") and file.endswith(".txt")]
data = []

for filename in file_names:
    print(filename)
    extract(directory_path + filename, data)

# conversion
data = [
    [int(value) if index in range(3) else float(value) for index, value in enumerate(row)]
    for row in data
]


# 加核数
data = [[core_num] + row for row in data]
df = pd.DataFrame(data, columns=['cores', 'NX', 'NY', 'NZ', 'Time', 'Gflops'])
df

HPCG-Benchmark_3.1_2023-11-11_15-55-20.txt
HPCG-Benchmark_3.1_2023-11-11_16-33-55.txt
HPCG-Benchmark_3.1_2023-11-12_15-22-11.txt
HPCG-Benchmark_3.1_2023-11-12_15-22-42.txt
HPCG-Benchmark_3.1_2023-11-12_15-26-33.txt
HPCG-Benchmark_3.1_2023-11-12_15-59-31.txt
HPCG-Benchmark_3.1_2023-11-12_16-03-01.txt
HPCG-Benchmark_3.1_2023-11-12_16-32-04.txt
HPCG-Benchmark_3.1_2023-11-12_16-42-06.txt
HPCG-Benchmark_3.1_2023-11-12_17-03-47.txt
HPCG-Benchmark_3.1_2023-11-12_17-26-59.txt
HPCG-Benchmark_3.1_2023-11-12_17-38-07.txt
HPCG-Benchmark_3.1_2023-11-12_18-11-49.txt
HPCG-Benchmark_3.1_2023-11-12_18-47-54.txt
HPCG-Benchmark_3.1_2023-11-12_19-11-36.txt
HPCG-Benchmark_3.1_2023-11-12_20-45-22.txt
HPCG-Benchmark_3.1_2023-11-12_21-31-21.txt
HPCG-Benchmark_3.1_2023-11-12_22-05-06.txt
HPCG-Benchmark_3.1_2023-11-12_22-39-34.txt
HPCG-Benchmark_3.1_2023-11-12_23-15-23.txt
HPCG-Benchmark_3.1_2023-11-13_12-00-10.txt
HPCG-Benchmark_3.1_2023-11-13_15-05-10.txt
HPCG-Benchmark_3.1_2023-11-13_15-36-13.txt
HPCG-Benchm

,cores,NX,NY,NZ,Time,Gflops
0,24,104,104,104,1879.53,4.71693
1,24,24,104,104,1954.35,4.06886
2,24,24,104,104,2156.92,3.77031
3,24,24,104,104,1896.82,8.85285
4,24,24,104,104,1765.05,3.62401
...,...,...,...,...,...,...
383,24,16,16,16,1754.74,17.94090
384,24,16,16,16,1986.44,17.54220
385,24,16,16,16,2087.60,17.66670
386,24,16,16,16,1967.72,17.66220


In [4]:
# 存储到数据库
df.to_sql('hpcg', conn, if_exists='append', index=False)

388

In [5]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM hpcg ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

[(24, 16, 16, 16, 1914.64, 18.1416)]

In [6]:
# 关闭游标和连接
cursor.close()

# 关闭数据库连接
conn.close()